## 코로나 확진자 수 구하기

In [1]:
import requests
from bs4 import BeautifulSoup
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import urllib
from urllib.parse import urlencode, quote_plus
import requests
from bs4 import BeautifulSoup
import json
import json
from pandas import json_normalize

In [2]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
#ServiceKey = '%2FVg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0%2BIYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA%3D%3D'
open_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'


In [3]:
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '10',
                               quote_plus('startCreateDt') : '20200110',
                               quote_plus('endCreateDt') : '20210831' })

open_url_get = open_url + queryParams

In [4]:
res = requests.get(open_url_get)
datas = BeautifulSoup(res.content,"html.parser").find_all("item")
#info = json.loads(res.content)
#json_normalize(info['data']["list"])

In [5]:
일자 = [] ; 사망자수 = [] ; 시도명 = [] ; 확진자수 = [] ; 지역발생수 = [] ; 발생률_10만명당 = []

In [6]:
for data in datas:
    일자.append(data.find("createdt").get_text())
    사망자수.append(data.find("deathcnt").get_text())
    시도명.append(data.find("gubun").get_text())
    확진자수.append(data.find("incdec").get_text()) 
    #지역발생수.append(data.find("localocccnt").get_text()) -> 지역 발생은 2020 4 9 까지 밖에 없음
    발생률_10만명당.append(data.find("qurrate").get_text())

In [7]:
df = pd.DataFrame(columns = ["일자","사망자수","시도명","확진자수","10만명당 발생률"])

df["일자"] = 일자
df["사망자수"] = 사망자수
df["시도명"] = 시도명
df["확진자수"] = 확진자수
df["10만명당 발생률"] = 발생률_10만명당

In [8]:
jeju = df[df["시도명"] == "제주"].reset_index(drop=True)
jeju["일자"] = jeju["일자"].apply(lambda x : x[:10])
jeju.loc[485,"10만명당 발생률"] = 0
# 3월 2일 확진자 0로 만들어주기

3월 2일 이전에는 확진자가 없으니 전부다 0으로 만들어주기

In [11]:
jeju.tail(2)

,일자,사망자수,시도명,확진자수,10만명당 발생률
546,2020-03-04,0,제주,0,0.4
547,2020-03-02,0,제주,0,NULL


In [12]:
date = pd.date_range("2018-01-01","2020-03-01").astype(str)
jeju_3월2일이전 = pd.DataFrame(columns=["날짜","사망자수","시도명","확진자수","10만명당 발생률"])
jeju_3월2일이전.columns = ["일자","사망자수","시도명","확진자수","10만명당 발생률"]
jeju_3월2일이전["일자"] = date
jeju_3월2일이전["시도명"] = "제주"
jeju_3월2일이전["사망자수"] = 0
jeju_3월2일이전["확진자수"] = 0
jeju_3월2일이전["10만명당 발생률"] = 0

In [13]:
jeju_3월2일이전.head(2)

,일자,사망자수,시도명,확진자수,10만명당 발생률
0,2018-01-01,0,제주,0,0
1,2018-01-02,0,제주,0,0


In [14]:
jeju_확진자수 = pd.concat([jeju,jeju_3월2일이전],axis=0,
                      ignore_index=True).sort_values("일자").drop("시도명",axis=1).reset_index(drop=True)

In [15]:
jeju_확진자수["일자"] = pd.to_datetime(jeju_확진자수["일자"],format="%Y-%m-%d")
jeju_확진자수["년"] = jeju_확진자수["일자"].apply(lambda x : x.year)
jeju_확진자수["월"] = jeju_확진자수["일자"].apply(lambda x : x.month)
jeju_확진자수["일"] = jeju_확진자수["일자"].apply(lambda x : x.day)

In [18]:
jeju_확진자수.to_csv("제주도 일별확진자수 전처리완료.csv",index=False,encoding="utf-8-sig")

## 제주 예상 승객

In [9]:
open_url = "http://openapi.airport.co.kr/service/rest/dailyExpectPassenger/dailyExpectPassenger"
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="

In [10]:
def get_예상승객(date):
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                   quote_plus('schDate') : date,
                                   quote_plus('schAirport') : 'CJU',
                                   quote_plus('serviceKey') : ServiceKey })
    
    open_url_get = open_url + queryParams
    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    
    시간대 = [] ; 예상승객수 = [] ; 단체승객수 = []

    for data in datas:
        시간대.append(data.find("hh").get_text())
        예상승객수.append(data.find("pct").get_text())
        단체승객수.append(data.find("pcg").get_text())
    
    res_df = pd.DataFrame(columns=["시간대","예상승객수","단체승객수"])
    res_df["시간대"] = 시간대 ; res_df["예상승객수"] = 예상승객수; res_df["단체승객수"] = 단체승객수
    
    return res_df

In [11]:
date = pd.date_range("2018-01-01","2021-08-31")
date_str = []
for i in date.strftime("%Y-%m-%d"):
    date_str.append(i.replace("-",""))

2018년부터 2021년까지 날짜 하나하나 씩 넣어주면서 get_예상승객 함수 돌리기
- date_str : "20180101" ~ "20210801" 날짜데이터 문자열 형식으로

In [12]:
result_lst = []

for date in date_str:
    result_lst.append(get_예상승객(date))

각 날짜별 데이터프레임에 날짜 넣어주기 ex) "20180101"

In [13]:
for i in range(len(result_lst)):
    result_lst[i]["날짜"] = date_str[i]

In [14]:
제주공항_예상승객수 = pd.concat(result_lst,axis=0)

In [16]:
제주공항_예상승객수.tail(2)

,시간대,예상승객수,단체승객수,날짜
8,10,1111,0,20210831
9,11,1662,0,20210831


In [32]:
#제주공항_예상승객수.to_csv("제주공항_예상승객수.csv",index=False,encoding="utf-8-sig")

In [33]:
#제주공항_예상승객수 = pd.read_csv("제주공항_예상승객수.csv").sort_values(["일자","시간대"])


날짜 데이터 생성해주기

In [34]:
제주공항_예상승객수.columns = ["시간대","예상승객수","단체승객수","일자"]
제주공항_예상승객수["일자"] = pd.to_datetime(제주공항_예상승객수["일자"],format="%Y-%m-%d")
제주공항_예상승객수["년"] = 제주공항_예상승객수["일자"].apply(lambda x : x.year)
제주공항_예상승객수["월"] = 제주공항_예상승객수["일자"].apply(lambda x : x.month)
제주공항_예상승객수["일"] = 제주공항_예상승객수["일자"].apply(lambda x : x.day)

제주공항_예상승객수["예상승객수"] = 제주공항_예상승객수["예상승객수"].astype(int)

### 일별로 데이터 정리

In [35]:
제주공항_예상승객수_일별 = 제주공항_예상승객수.groupby(["일자","년",
                                    "월","일"]).agg({"예상승객수":["sum","std"]}).reset_index()

In [36]:
제주공항_예상승객수_일별.head(3)

일자     년  월  일  예상승객수             
                            sum          std
0 2018-01-01  2018  1  1  16217  1134.862698
1 2018-01-02  2018  1  2  16143  1039.262351
2 2018-01-03  2018  1  3  12486  1004.534408

### 월별로 데이터 정리

In [37]:
제주공항_예상승객수_월별 = 제주공항_예상승객수.groupby(["년","월"]).agg({"예상승객수":["sum","std"]}).reset_index()

In [38]:
제주공항_예상승객수_월별.head(3)

년  월   예상승객수            
               sum         std
0  2018  1  394747  873.709080
1  2018  2  326065  821.357614
2  2018  3  339005  816.612698

### 월별데이터와 일별 데이터 결합
- 제주공항_예상승객수_월별
- 제주공항_예상승객수_일별
- 제주공항_예상승객수_일별_월별

In [39]:
제주공항_예상승객수_일별_월별 = pd.merge(제주공항_예상승객수_일별,제주공항_예상승객수_월별,
                           on = ["년","월"],how = "inner")
제주공항_예상승객수_일별_월별.columns = ["일자","년","월","일","일별예상승객총합",
                           "일별예상승객편차","월별예상승객총합","월별예상승객편차"]

In [40]:
제주공항_예상승객수_일별_월별["일별예상승객비율"] = ( 제주공항_예상승객수_일별_월별["일별예상승객총합"] /
                                제주공항_예상승객수_일별_월별["월별예상승객총합"] )

In [41]:
제주공항_예상승객수_일별_월별.to_csv("제주공항_예상승객수 전처리완료.csv",index=False,encoding="utf-8-sig")